In [1]:
from typing import Any, Callable, Dict, Optional
from numpy import ndarray
from typing import Any, Dict, Optional, Tuple
from numpy import hstack, ndarray, sqrt, vstack, zeros
from numpy.linalg import cholesky
from numpy.linalg import LinAlgError
from scipy.sparse import csc_matrix
from typing import Optional
from warnings import warn
from ecos import solve
from numpy import ndarray
from scipy import sparse
import numpy as np


In [2]:

def convert_to_socp( P, q, G, h)-> Tuple[ndarray, ndarray, ndarray, Dict[str, Any]]:

    n = P.shape[1]  # dimension of QP variable
    c_socp = hstack([zeros(n), 1])  # new SOCP variable stacked as [x, t]
    try:
        L = cholesky(P)
    except LinAlgError as e:
        error = str(e)
        if "not positive definite" in error:
            raise ValueError("matrix P is not positive definite") from e
        raise e  # other linear algebraic error

    scale = 1.0 / sqrt(2)
    G_quad = vstack(
        [
            scale * hstack([q, -1.0]),
            hstack([-L.T, zeros((L.shape[0], 1))]),
            scale * hstack([-q, +1.0]),
        ]
    )
    h_quad = hstack([scale, zeros(L.shape[0]), scale])

    dims: Dict[str, Any] = {"q": [L.shape[0] + 2]}
    G_socp = vstack([hstack([G, zeros((G.shape[0], 1))]), G_quad])
    h_socp = hstack([h, h_quad])
    dims["l"] = G.shape[0]

    G_socp = csc_matrix(G_socp)
    return c_socp, G_socp, h_socp, dims


def ecos_solve_qp( P, q, G, h, A, initvals: Optional[ndarray] = None,verbose: bool = False,
                 ) -> Optional[ndarray]:
      
    c_socp, G_socp, h_socp, dims = convert_to_socp(P, q, G, h) 
    A_socp = sparse.hstack([A, sparse.csc_matrix((A.shape[0], 1))], format="csc")
    solution = solve(c_socp, G_socp, h_socp, dims, A_socp, b, verbose=verbose)
    flag = solution["info"]["exitFlag"]
    return solution["x"][:-1]



In [3]:
m = 5
n = 700
p = 5
P = np.loadtxt('P.txt')
q = np.loadtxt('q.txt')
G = np.loadtxt('G.txt')
h = np.loadtxt('h.txt')
A = np.loadtxt('A.txt')
b = np.loadtxt('b.txt')

x=ecos_solve_qp( P,q, G,h,A,b)
v=0.5*x.T@P@x+q.T@x
print(v)

-42.31408091332811
